In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

--- LogisticRegression ---

In [8]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr_cs= LogisticRegression(random_state=15, class_weight= 'balanced', max_iter= 1000)
lr_cs.fit(X_train, y_train)
y_pred_lr_cs= lr_cs.predict(X_test)
y_pred_prob_lr_cs= lr_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lr_cs, labels= lr_cs.classes_)
print(classification_report(y_test, y_pred_lr_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_lr_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr_cs))
print('AUC:', roc_auc_score(y_test, y_pred_lr_cs))

              precision    recall  f1-score   support

           0       0.97      0.75      0.84      8517
           1       0.25      0.81      0.39       912

    accuracy                           0.75      9429
   macro avg       0.61      0.78      0.61      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.25318197454420366
Accuracy: 0.7510870717997666
AUC: 0.7760577915376676


--- DecisionTreeClassifier ---

In [9]:
from sklearn.tree import DecisionTreeClassifier
dt_cs= DecisionTreeClassifier(class_weight= 'balanced', random_state= 15)
dt_cs.fit(X_train, y_train)
y_pred_dt_cs= dt_cs.predict(X_test)
y_pred_prob_dt_cs= dt_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_dt_cs, labels= dt_cs.classes_)
print(classification_report(y_test, y_pred_dt_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_dt_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt_cs))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      8517
           1       0.23      0.22      0.22       912

    accuracy                           0.85      9429
   macro avg       0.57      0.57      0.57      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.22591362126245848
Accuracy: 0.8507795100222717
AUC: 0.5709979035736577


--- Tuned - DecisionTreeClassifier ---

In [10]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt_cs= GridSearchCV(estimator= dt_cs, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt_cs.fit(X_train, y_train)

print("Best Parameters:", gs_dt_cs.best_params_)
print("Best Precision Score:", gs_dt_cs.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.24502888724061084


In [11]:
tuned_dt_cs= gs_dt_cs.best_estimator_
y_pred_tuned_dt_cs= tuned_dt_cs.predict(X_test)
y_pred_prob_tuned_dt_cs= tuned_dt_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt_cs, labels= tuned_dt_cs.classes_)
print(classification_report(y_test, y_pred_tuned_dt_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt_cs))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      8517
           1       0.24      0.24      0.24       912

    accuracy                           0.85      9429
   macro avg       0.58      0.58      0.58      9429
weighted avg       0.85      0.85      0.85      9429

Precision: 0.24282560706401765
Accuracy: 0.8538551277972214
AUC: 0.5805823852810374


--- SVM ---

In [12]:
from sklearn.svm import SVC
svc_cs= SVC(kernel= 'rbf',probability= True, class_weight= 'balanced', gamma= 1, random_state= 15)
svc_cs.fit(X_train, y_train)
y_pred_svc_cs= svc_cs.predict(X_test)
y_pred_prob_svc_cs= svc_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_svc_cs, labels= svc_cs.classes_)
print(classification_report(y_test, y_pred_svc_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_svc_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc_cs))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      8517
           1       0.15      0.03      0.05       912

    accuracy                           0.89      9429
   macro avg       0.53      0.51      0.50      9429
weighted avg       0.83      0.89      0.86      9429

Precision: 0.15204678362573099
Accuracy: 0.8906564853112737
AUC: 0.6954538420578863


--- Random Forest ---

In [13]:
from sklearn.ensemble import RandomForestClassifier
rf_cs= RandomForestClassifier(class_weight= 'balanced', random_state= 15)
rf_cs.fit(X_train, y_train)
y_pred_rf_cs= rf_cs.predict(X_test)
y_pred_prob_rf_cs= rf_cs.predict_proba(X_test)[:,1]

In [14]:
cm= confusion_matrix(y_test, y_pred_rf_cs, labels= rf_cs.classes_)
print(classification_report(y_test, y_pred_rf_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_rf_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf_cs))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.39      0.06      0.10       912

    accuracy                           0.90      9429
   macro avg       0.65      0.52      0.52      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.38848920863309355
Accuracy: 0.8999893944214656
AUC: 0.8204212704621716


In [15]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf_cs= GridSearchCV(estimator= rf_cs, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf_cs.fit(X_train, y_train)
print("Best Parameters:", gs_tuned_rf_cs.best_params_)
print("Best Precision Score:", gs_tuned_rf_cs.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.45930032763163126


In [16]:
tuned_rf_cs= gs_tuned_rf_cs.best_estimator_
y_pred_tuned_rf_cs= tuned_rf_cs.predict(X_test)
y_pred_prob_tuned_rf_cs= tuned_rf_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf_cs, labels= tuned_rf_cs.classes_)
print(classification_report(y_test, y_pred_tuned_rf_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf_cs))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      8517
           1       0.38      0.05      0.09       912

    accuracy                           0.90      9429
   macro avg       0.64      0.52      0.52      9429
weighted avg       0.86      0.90      0.86      9429

Precision: 0.3790322580645161
Accuracy: 0.9000954502068088
AUC: 0.8253678401710511


--- AdaBoost ---

In [17]:
from sklearn.ensemble import AdaBoostClassifier
ada_cs= AdaBoostClassifier(random_state=15)
ada_cs.fit(X_train, y_train, sample_weight= (y_train== 0).astype(int) + (y_train== 1).astype(int) * 2)
y_pred_ada_cs= ada_cs.predict(X_test)
y_pred_prob_ada_cs= ada_cs.predict_proba(X_test)[:,1]

In [18]:
cm= confusion_matrix(y_test, y_pred_ada_cs, labels= ada_cs.classes_)
print(classification_report(y_test, y_pred_ada_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_ada_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada_cs))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      8517
           1       0.45      0.31      0.37       912

    accuracy                           0.90      9429
   macro avg       0.69      0.63      0.65      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.45032051282051283
Accuracy: 0.8967016650758299
AUC: 0.8531471628466494


In [19]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada_cs= GridSearchCV(estimator= ada_cs, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada_cs.fit(X_train, y_train, sample_weight=(y_train == 0).astype(int) + (y_train == 1).astype(int) * 2)
print("Best Parameters:", gs_ada_cs.best_params_)
print("Best Precision Score:", gs_ada_cs.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 50}
Best Precision Score: 0.4252253078879136


In [20]:
tuned_ada_cs= gs_ada_cs.best_estimator_
y_pred_tuned_ada_cs= tuned_ada_cs.predict(X_test)
y_pred_prob_tuned_ada_cs= tuned_ada_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada_cs, labels= tuned_ada_cs.classes_)
print(classification_report(y_test, y_pred_tuned_ada_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada_cs))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      8517
           1       0.45      0.31      0.37       912

    accuracy                           0.90      9429
   macro avg       0.69      0.63      0.65      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.45032051282051283
Accuracy: 0.8967016650758299
AUC: 0.8531471628466494


--- GradientBoosting ---

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
grb_cs= GradientBoostingClassifier(random_state= 15)
grb_cs.fit(X_train, y_train, sample_weight= (y_train== 0).astype(int) + (y_train== 1).astype(int) * 2)
y_pred_grb_cs= grb_cs.predict(X_test)
y_pred_prob_grb_cs= grb_cs.predict_proba(X_test)[:,1]


cm= confusion_matrix(y_test, y_pred_grb_cs, labels= grb_cs.classes_)
print(classification_report(y_test, y_pred_grb_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_grb_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb_cs))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      8517
           1       0.46      0.29      0.36       912

    accuracy                           0.90      9429
   macro avg       0.69      0.63      0.65      9429
weighted avg       0.88      0.90      0.89      9429

Precision: 0.4570446735395189
Accuracy: 0.897974334499947
AUC: 0.8540342560493049


In [22]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb_cs= GridSearchCV(estimator= grb_cs, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb_cs.fit(X_train, y_train, sample_weight= (y_train== 0).astype(int) + (y_train== 1).astype(int) * 2)

print("Best Parameters:", gs_grb_cs.best_params_)
print("Best Precision Score:", gs_grb_cs.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.01}
Best Precision Score: 0.7094227504244481


In [23]:
tuned_grb_cs= gs_grb_cs.best_estimator_
y_pred_tuned_grb_cs= tuned_grb_cs.predict(X_test)
y_pred_prob_tuned_grb_cs= tuned_grb_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb_cs, labels= tuned_grb_cs.classes_)
print(classification_report(y_test, y_pred_tuned_grb_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb_cs))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      8517
           1       0.58      0.02      0.03       912

    accuracy                           0.90      9429
   macro avg       0.74      0.51      0.49      9429
weighted avg       0.87      0.90      0.86      9429

Precision: 0.5769230769230769
Accuracy: 0.9037013469084738
AUC: 0.8301535473943753


--- XGB ---

In [24]:
from xgboost import XGBClassifier
xgb_cs= XGBClassifier(scale_pos_weight= (y_train== 0).sum() / (y_train== 1).sum(), random_state= 15)
xgb_cs.fit(X_train, y_train)
y_pred_xgb_cs= xgb_cs.predict(X_test)
y_pred_prob_xgb_cs= xgb_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb_cs, labels= xgb_cs.classes_)
print(classification_report(y_test, y_pred_xgb_cs, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb_cs, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb_cs))

              precision    recall  f1-score   support

           0       0.96      0.80      0.87      8517
           1       0.26      0.65      0.37       912

    accuracy                           0.79      9429
   macro avg       0.61      0.73      0.62      9429
weighted avg       0.89      0.79      0.82      9429

Precision: 0.2624394539850286
Accuracy: 0.7888429313819069
AUC: 0.8139428058228229


In [25]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb_cs= GridSearchCV(estimator= xgb_cs, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb_cs.fit(X_train, y_train)

print("Best Parameters:", gs_xgb_cs.best_params_)
print("Best Precision Score:", gs_xgb_cs.best_score_)

tuned_xgb_cs= gs_xgb_cs.best_estimator_
y_pred_tuned_xgb_cs= tuned_xgb_cs.predict(X_test)
y_pred_prob_tuned_xgb_cs= tuned_xgb_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb_cs, labels= tuned_xgb_cs.classes_)
print(classification_report(y_test, y_pred_tuned_xgb_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb_cs))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 1.0}
Best Precision Score: 0.24946840139333643
              precision    recall  f1-score   support

           0       0.93      0.86      0.89      8517
           1       0.25      0.44      0.32       912

    accuracy                           0.82      9429
   macro avg       0.59      0.65      0.61      9429
weighted avg       0.87      0.82      0.84      9429

Precision: 0.250155183116077
Accuracy: 0.8179022165659137
AUC: 0.7512845825377109


--- LGBM ---

In [26]:
from lightgbm import LGBMClassifier
lgm_cs= LGBMClassifier(class_weight='balanced', random_state= 15)
lgm_cs.fit(X_train, y_train)
y_pred_lgm_cs= lgm_cs.predict(X_test)
y_pred_prob_lgm_cs= lgm_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm_cs, labels= lgm_cs.classes_)
print(classification_report(y_test, y_pred_lgm_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_lgm_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm_cs))

[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006451 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
              precision    recall  f1-score   support

           0       0.97      0.75      0.84      8517
           1       0.25      0.79      0.38       912

    accuracy                           0.75      9429
   macro avg       0.61      0.77      0.61      9429
weighted avg       0.90      0.75      0.80      9429

Precision: 0.24843858431644691
Accuracy: 0.7494962350196204
AUC: 0.8469875264949976


In [27]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm_cs= GridSearchCV(estimator= lgm_cs, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm_cs.fit(X_train, y_train)

print("Best Parameters:", gs_lgm_cs.best_params_)
print("Best Precision Score:", gs_lgm_cs.best_score_)

tuned_lgm_cs= gs_lgm_cs.best_estimator_
y_pred_tuned_lgm_cs= tuned_lgm_cs.predict(X_test)
y_pred_prob_tuned_lgm_cs= tuned_lgm_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm_cs, labels= tuned_lgm_cs.classes_)
print(classification_report(y_test, y_pred_tuned_lgm_cs, zero_division= 0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm_cs, zero_division= 0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm_cs))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Number of positive: 3649, number of negative: 34065
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005493 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 251
[LightGBM] [Info] Number of data points in the train set: 37714, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Best Parameters: {'learning_rate': 1.0, 'min_data_in_leaf': 50, 'num_leaves': 127, 'reg_alpha': 0.1}
Best Precision Score: 0.2738656824369

--- CatBoost ---

In [28]:
from catboost import CatBoostClassifier
cat_cs= CatBoostClassifier(class_weights= [1, (y_train== 0).sum() / (y_train== 1).sum()], random_state=15, silent=True)
cat_cs.fit(X_train, y_train)
y_pred_cat_cs= cat_cs.predict(X_test)
y_pred_prob_cat_cs= cat_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_cat_cs, labels= cat_cs.classes_)
print(classification_report(y_test, y_pred_cat_cs, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat_cs, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat_cs))

              precision    recall  f1-score   support

           0       0.96      0.79      0.87      8517
           1       0.27      0.70      0.39       912

    accuracy                           0.79      9429
   macro avg       0.61      0.75      0.63      9429
weighted avg       0.89      0.79      0.82      9429

Precision: 0.2676409185803758
Accuracy: 0.7852370346802418
AUC: 0.8361629424329874


In [29]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat_cs= GridSearchCV(estimator= cat_cs, param_grid= params_CatBoost, cv= 5, scoring='precision', n_jobs= -1, verbose= 2)
gs_cat_cs.fit(X_train, y_train)

print("Best Parameters:", gs_cat_cs.best_params_)
print("Best Precision Score:", gs_cat_cs.best_score_)

tuned_cat_cs= gs_cat_cs.best_estimator_
y_pred_tuned_cat_cs= tuned_cat_cs.predict(X_test)
y_pred_prob_tuned_cat_cs= tuned_cat_cs.predict_proba(X_test)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat_cs, labels= tuned_cat_cs.classes_)
print(classification_report(y_test, y_pred_tuned_cat_cs, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat_cs, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat_cs))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat_cs))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best Parameters: {'depth': 10, 'iterations': 100, 'learning_rate': 1}
Best Precision Score: 0.28813063260922284
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      8517
           1       0.27      0.29      0.28       912

    accuracy                           0.86      9429
   macro avg       0.60      0.60      0.60      9429
weighted avg       0.86      0.86      0.86      9429

Precision: 0.27395833333333336
Accuracy: 0.8572489129282003
AUC: 0.7462397187050049


--- Visualisation ---

In [30]:
print('Logistic Regression Precision:', precision_score(y_test, y_pred_lr_cs, zero_division= 0))
print('Decision Tree Precision:', precision_score(y_test, y_pred_dt_cs, zero_division= 0))
print('Tuned Decision Tree Precision:', precision_score(y_test, y_pred_tuned_dt_cs, zero_division= 0))
print('SVM Precision:', precision_score(y_test, y_pred_svc_cs, zero_division= 0))
print('Random Forest Precision:', precision_score(y_test, y_pred_rf_cs, zero_division= 0))
print('Tuned Random Forest Precision:', precision_score(y_test, y_pred_tuned_rf_cs, zero_division= 0))
print('AdaBoost Precision:', precision_score(y_test, y_pred_ada_cs, zero_division= 0))
print('Tuned AdaBoost Precision:', precision_score(y_test, y_pred_tuned_ada_cs, zero_division= 0))
print('GradientBoosting Precision:', precision_score(y_test, y_pred_grb_cs, zero_division= 0))
print('Tuned GradientBoosting Precision:', precision_score(y_test, y_pred_tuned_grb_cs, zero_division= 0))
print('XGB Precision:', precision_score(y_test, y_pred_xgb_cs, zero_division= 0))
print('Tuned XGB Precision:', precision_score(y_test, y_pred_tuned_xgb_cs, zero_division= 0))
print('LGBM Precision:', precision_score(y_test, y_pred_lgm_cs, zero_division= 0))
print('Tuned LGBM Precision:', precision_score(y_test, y_pred_tuned_lgm_cs, zero_division= 0))
print('CatBoost Precision:', precision_score(y_test, y_pred_cat_cs, zero_division= 0))
print('Tuned CatBoost Precision:', precision_score(y_test, y_pred_tuned_cat_cs, zero_division= 0))

Logistic Regression Precision: 0.25318197454420366
Decision Tree Precision: 0.22591362126245848
Tuned Decision Tree Precision: 0.24282560706401765
SVM Precision: 0.15204678362573099
Random Forest Precision: 0.38848920863309355
Tuned Random Forest Precision: 0.3790322580645161
AdaBoost Precision: 0.45032051282051283
Tuned AdaBoost Precision: 0.45032051282051283
GradientBoosting Precision: 0.4570446735395189
Tuned GradientBoosting Precision: 0.5769230769230769
XGB Precision: 0.2624394539850286
Tuned XGB Precision: 0.250155183116077
LGBM Precision: 0.24843858431644691
Tuned LGBM Precision: 0.26099706744868034
CatBoost Precision: 0.2676409185803758
Tuned CatBoost Precision: 0.27395833333333336


In [31]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr_cs, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt_cs, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt_cs, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc_cs, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf_cs, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf_cs, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada_cs, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada_cs, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb_cs, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb_cs, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb_cs, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb_cs, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm_cs, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm_cs, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat_cs, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat_cs, zero_division= 0)
                  }

cs_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
cs_precision= cs_precision.sort_values(by= 'Precision Score', ascending=False)
print(cs_precision)

                                Model  Precision Score
9   Tuned GradientBoosting Precision:         0.576923
8         GradientBoosting Precision:         0.457045
6                 AdaBoost Precision:         0.450321
7           Tuned AdaBoost Precision:         0.450321
4            Random Forest Precision:         0.388489
5      Tuned Random Forest Precision:         0.379032
15          Tuned CatBoost Precision:         0.273958
14                CatBoost Precision:         0.267641
10                     XGB Precision:         0.262439
13              Tuned LGBM Precision:         0.260997
0      Logistic Regression Precision:         0.253182
11               Tuned XGB Precision:         0.250155
12                    LGBM Precision:         0.248439
2      Tuned Decision Tree Precision:         0.242826
1            Decision Tree Precision:         0.225914
3                      SVM Precision:         0.152047


In [32]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr_cs),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt_cs),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt_cs),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc_cs),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf_cs),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf_cs),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada_cs),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada_cs),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb_cs),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb_cs),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb_cs),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb_cs),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm_cs),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm_cs),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat_cs),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat_cs)
                  }

cs_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
cs_accuracy= cs_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(cs_accuracy)

                               Model  Accuracy Score
9   Tuned GradientBoosting Accuracy:        0.903701
5      Tuned Random Forest Accuracy:        0.900095
4            Random Forest Accuracy:        0.899989
8         GradientBoosting Accuracy:        0.897974
6                 AdaBoost Accuracy:        0.896702
7           Tuned AdaBoost Accuracy:        0.896702
3                      SVM Accuracy:        0.890656
15          Tuned CatBoost Accuracy:        0.857249
2      Tuned Decision Tree Accuracy:        0.853855
13              Tuned LGBM Accuracy:        0.851416
1            Decision Tree Accuracy:        0.850780
11               Tuned XGB Accuracy:        0.817902
10                     XGB Accuracy:        0.788843
14                CatBoost Accuracy:        0.785237
0      Logistic Regression Accuracy:        0.751087
12                    LGBM Accuracy:        0.749496


In [33]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr_cs),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt_cs),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt_cs),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc_cs),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf_cs),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf_cs),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada_cs),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada_cs),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb_cs),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb_cs),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb_cs),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb_cs),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm_cs),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm_cs),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat_cs),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat_cs)
                  }

cs_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
cs_auc= cs_auc.sort_values(by= 'AUC Score', ascending=False)
print(cs_auc)

                          Model  AUC Score
8         GradientBoosting AUC:   0.854034
0      Logistic Regression AUC:   0.853933
6                 AdaBoost AUC:   0.853147
7           Tuned AdaBoost AUC:   0.853147
12                    LGBM AUC:   0.846988
14                CatBoost AUC:   0.836163
9   Tuned GradientBoosting AUC:   0.830154
5      Tuned Random Forest AUC:   0.825368
4            Random Forest AUC:   0.820421
10                     XGB AUC:   0.813943
13              Tuned LGBM AUC:   0.761834
11               Tuned XGB AUC:   0.751285
15          Tuned CatBoost AUC:   0.746240
3                      SVM AUC:   0.695454
2      Tuned Decision Tree AUC:   0.580582
1            Decision Tree AUC:   0.570998
